In [1]:
import pandas as pd
import numpy as np
import keras

from keras.models import Sequential
from keras.layers import Dense

# load the concrete_data dataset
concrete_data = pd.read_csv('https://cocl.us/concrete_data')

# verify that the data was loaded correctly
concrete_data.head()

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [2]:
# split the concrete_data set into predictors (inputs) and target (output)
predictors = concrete_data.drop(columns=['Strength'])
target = concrete_data['Strength']

n_cols = predictors.shape[1]

In [3]:
def regression_model():
    model = Sequential()
    model.add(Dense(10, activation = 'relu', input_shape=(n_cols,)))
    model.add(Dense(1)) ##
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')
    
    return model

model = regression_model()

In [4]:
from sklearn.model_selection import train_test_split

predictors_train, predictors_test, target_train, target_test = train_test_split(
    predictors, target, test_size = 0.3, random_state = 71)

predictors_train.shape

(721, 8)

In [5]:
model.fit(predictors_train, target_train, epochs=50)

Epoch 1/50
721/721 [==============================] - 1s 846us/step - loss: 95846.8588
Epoch 2/50
721/721 [==============================] - 0s 417us/step - loss: 34352.6652
Epoch 3/50
721/721 [==============================] - 0s 328us/step - loss: 10697.8074
Epoch 4/50
721/721 [==============================] - 0s 528us/step - loss: 3612.4757
Epoch 5/50
721/721 [==============================] - 0s 333us/step - loss: 2461.9844
Epoch 6/50
721/721 [==============================] - 0s 420us/step - loss: 2216.4471
Epoch 7/50
721/721 [==============================] - 0s 421us/step - loss: 1999.2483
Epoch 8/50
721/721 [==============================] - 0s 443us/step - loss: 1801.0665
Epoch 9/50
721/721 [==============================] - 0s 410us/step - loss: 1615.5443
Epoch 10/50
721/721 [==============================] - 0s 364us/step - loss: 1447.6008
Epoch 11/50
721/721 [==============================] - 0s 522us/step - loss: 1288.7757
Epoch 12/50
721/721 [============================

In [6]:
from sklearn.metrics import mean_squared_error

In [7]:
predictions_test = model.predict(predictors_test)
print("Mean squared error on test data is %.3f" % 
      (mean_squared_error(target_test, predictions_test)))

Mean squared error on test data is 133.446


In [8]:
def evaluate_model(create_model_func, predictors, targets, epochs = 50):
    mean_squared_errors =[]
    for i in range(50):
        #Road the model
        model = create_model_func()
        
        #Split data into train and test
        predictors_train, predictors_test, target_train, target_test = train_test_split(
        predictors, target, test_size=0.3, random_state = 70)
        
        #Train 50 epochs
        model.fit(predictors_train, target_train, epochs = epochs, verbose=0)
        
        #Measure the mse and add this to the list
        predictions_test = model.predict(predictors_test)
        mse = mean_squared_error(target_test, predictions_test)
        mean_squared_errors.append(mse)
        print('...', end='')
    print('Done!')
    
    return np.mean(mean_squared_errors), np.std(mean_squared_errors)

In [9]:
#Part B
#Normalize the data
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [10]:
mean_mse, std_mse = evaluate_model(regression_model, predictors_norm, target)
#REPORT THE MEAN AND STD OF THE MEAN-SQUARED-ERROR LOSS
print("Mean squared errors for 50 epoch regression models on normalized data: mean = %.3f, std = %.3f"
     % (mean_mse, std_mse))

......................................................................................................................................................Done!
Mean squared errors for 50 epoch regression models on normalized data: mean = 367.056, std = 102.510
